In [ ]:
"""This ipynb helps inspect mirror and crop effect on image and annotation"""

In [ ]:
import numpy as np
from PIL import Image
import os
import scipy.io
from prostate_dataset import ProstateDataset

In [ ]:
def _crop_split_image(img_org, annotation_org, \
                                     height = 200, width = 200, \
                                     image_id = 0, save = False):
        
        imgheight, imgwidth, _ = img_org.shape
        annheight, annwidth = annotation_org.shape
        assert imgwidth == annwidth and imgheight == annheight, "The image and annotation size don't match!"
        
        img = []
        ann = []
        
        current_image = 0
        for i in range(0, imgheight - 101, 100):
            for j in range(0, imgwidth - 101, 100):
                img_temp = img_org[i : i + height, j : j + width, :]
                img.append(img_temp)
                ann_temp = annotation_org[i : i + height, j : j + width]
                ann.append(ann_temp)
                if save:
                    ann_dict = {'ATmask': ann_temp}
                    ann_filename = str(image_id).zfill(4) + '_' + str(current_image).zfill(4) + '_sementic.mat'
                    scipy.io.savemat('./cedars-224/masks_sementic_mod_144/' + \
                                     ann_filename, ann_dict)
                    img_tile = Image.fromarray(img_temp, 'RGB')
                    img_filename = str(image_id).zfill(4) + '_' + str(current_image).zfill(4) + '.jpg'
                    img_tile.save('./cedars-224/tiles_144/' + img_filename)
                    current_image += 1
        return img, ann

In [ ]:
def _image_padding(img_org, annotation_org):
        img_pad = np.lib.pad(img_org, ((50,50),(50,50), (0, 0)), 'symmetric')
        ann_pad = np.lib.pad(annotation_org, ((50, 50), (50, 50)), 'symmetric') 
        return img_pad, ann_pad

In [ ]:
data_path = '/data/wenyuan/Path_R_CNN/Data_Pre_Processing'
dataset_dir = os.path.join(data_path, 'cedars-224')
data = ProstateDataset(dataset_dir)

In [ ]:
image_id = 0
patch_num = 2
image_org = data.read_image(image_id, dir_name = '/tiles_144', patch_num = patch_num)

ann_org = data.read_original_ann(image_id, dir_name = '/masks_sementic_mod_144', patch_num = patch_num)
ann_decode = data.convert_mat_annotations_to_png(ann_org)
data.show_ann_png(ann_decode)